In [2]:
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation
from skimage import io, transform, color
import requests
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from typing import List

feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")

e:\Program Files\Python\lib\site-packages\transformers\models\segformer\feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
e:\Program Files\Python\lib\site-packages\transformers\models\segformer\image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm

def parse_csv(path: str = 'data/ADE20K/included_classes.csv', sep: str = ';'):
    csv = pd.read_csv(path, sep=sep)
    classes = {}
    for index, (name, inc) in enumerate(zip(csv['Name'], csv['Include'])):
        if inc:
            classes[name] = index
    return classes    
    
def clear_dataset():
    classes = parse_csv()
    stats = {key: 0 for key in classes.keys()}
    has_classes = []

    pixel_sums = 0
    for pimage in tqdm(glob('data/ADE20K/annotations/*.png')):
        im = io.imread(pimage, as_gray=True)
        flag = False
        for (val, key) in zip(classes.values(), classes.keys()):
            sums = np.sum(im == val)
            if sums > 0:
                flag = True
                stats[key] += sums
                pixel_sums += im.shape[0] * im.shape[1]
        if flag:
            has_classes.append(pimage)
            
    for key in stats.keys():
        stats[key] = stats[key] / pixel_sums
        
    return has_classes, stats
        
# im_list, stats = clear_dataset()

In [3]:
"""
150 classes
we need:
1. sky (2)
2. tree (4)
3. grass (9)
4. earth;ground (13)
5. mountain;mount (16)
6. plant;flora;plant;life (17)
7. sea (27)
8. 
"""

def image_to_mask(images, model, feature_extractor) -> List[np.ndarray]:
    inputs = feature_extractor(images=images, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    data: np.ndarray = np.array(logits.detach().numpy()).transpose((0, 2, 3, 1))
    image_mask = np.argmax(data, axis=-1, keepdims=True)
    return image_mask.astype(np.uint8)

def read_images(paths):
    return [io.imread(p) for p in paths]

classes = parse_csv()

files = glob('./data/lhq_256/images/*.png')
batch = 64
for i in tqdm(range(0, len(files), batch)):
    paths = files[i:i+batch]
    images = [io.imread(p) for p in paths]
    masks = image_to_mask(images, model, feature_extractor)
    for k in range(len(masks)):
        mask = transform.resize(masks[k], images[k].shape, preserve_range=True).astype(np.uint8)
        name = paths[k].replace('images', 'annotations')
        io.imsave(name, mask, check_contrast=False)
    
    # print(np.unique(mask.flatten()))

    # plt.axis('off')
    # plt.imshow(np.hstack([mask, image]))
    # plt.show()

    # if i > 3: 
    #     break


100%|██████████| 1407/1407 [4:46:18<00:00, 12.21s/it]  


0

In [1]:
import multiprocessing
import itertools
from typing import List
import numpy as np
import pandas as pd
from glob import glob

def parse_csv(path: str = 'data/ADE20K/included_classes.csv', sep: str = ';'):
    csv = pd.read_csv(path, sep=sep)
    classes = {}
    for index, (name, inc) in enumerate(zip(csv['Name'], csv['Include'])):
        if inc:
            classes[name] = index
    return classes    



if __name__ == '__main__':
    classes = parse_csv()
    transform_dataset(classes)
        


: 

In [2]:
import numpy as np

a = np.random.random((32, 32, 5))
np.argmax(a, axis=-1, keepdims=True)


array([[[1],
        [3],
        [2],
        ...,
        [2],
        [1],
        [0]],

       [[4],
        [3],
        [1],
        ...,
        [4],
        [2],
        [3]],

       [[0],
        [1],
        [0],
        ...,
        [4],
        [1],
        [1]],

       ...,

       [[0],
        [4],
        [0],
        ...,
        [2],
        [2],
        [1]],

       [[2],
        [0],
        [0],
        ...,
        [4],
        [2],
        [0]],

       [[3],
        [2],
        [0],
        ...,
        [3],
        [2],
        [1]]], dtype=int64)